In [4]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

IMG_SIZE = 512
# ------ Dette kodestykke er blevet lavet i samarbejde med ChatGPT ------
def load_images_from_folder(folder_path, label, img_size):
    data = []
    labels = []
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = load_img(img_path, target_size=(img_size, img_size))
        img_array = img_to_array(img) / 255.0
        data.append(img_array)
        labels.append(label)
    return data, labels
# ------ Dette kodestykke er blevet lavet i samarbejde med ChatGPT ------

# training data
train_is_path = "ice_cream/train/ice_cream"
train_not_path = "ice_cream/train/non_ice_cream"
train_data_is, labels_is = load_images_from_folder(train_is_path, 1, IMG_SIZE)
train_data_not, labels_not = load_images_from_folder(train_not_path, 0, IMG_SIZE)

X_train = np.array(train_data_is + train_data_not)
y_train = np.array(labels_is + labels_not)

# testing data
test_is_path = "ice_cream/test/ice_cream"
test_not_path = "ice_cream/test/non_ice_cream"
test_data_is, test_labels_is = load_images_from_folder(test_is_path, 1, IMG_SIZE)
test_data_not, test_labels_not = load_images_from_folder(test_not_path, 0, IMG_SIZE)

X_test = np.array(test_data_is + test_data_not)
y_test = np.array(test_labels_is + test_labels_not)


In [2]:
# Rapport generator funktioner
from time import time
import numpy as np
import sys

from sklearn import svm
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn import datasets

# from libitmal import dataloaders as itmaldataloaders # Needed for load of iris, moon and mnist

currmode="N/A" # GLOBAL var!

def SearchReport(model): 
    
    def GetBestModelCTOR(model, best_params):
        def GetParams(best_params):
            ret_str=""          
            for key in sorted(best_params):
                value = best_params[key]
                temp_str = "'" if str(type(value))=="<class 'str'>" else ""
                if len(ret_str)>0:
                    ret_str += ','
                ret_str += f'{key}={temp_str}{value}{temp_str}'  
            return ret_str          
        try:
            param_str = GetParams(best_params)
            return type(model).__name__ + '(' + param_str + ')' 
        except:
            return "N/A(1)"
        
    print("\nBest model set found on train set:")
    print()
    print(f"\tbest parameters={model.best_params_}")
    print(f"\tbest '{model.scoring}' score={model.best_score_}")
    print(f"\tbest index={model.best_index_}")
    print()
    print(f"Best estimator CTOR:")
    print(f"\t{model.best_estimator_}")
    print()
    try:
        print(f"Grid scores ('{model.scoring}') on development set:")
        means = model.cv_results_['mean_test_score']
        stds  = model.cv_results_['std_test_score']
        i=0
        for mean, std, params in zip(means, stds, model.cv_results_['params']):
            print("\t[%2d]: %0.3f (+/-%0.03f) for %r" % (i, mean, std * 2, params))
            i += 1
    except:
        print("WARNING: the random search do not provide means/stds")
    
    global currmode                
    assert "f1_micro"==str(model.scoring), f"come on, we need to fix the scoring to be able to compare model-fits! Your scoreing={str(model.scoring)}...remember to add scoring='f1_micro' to the search"   
    return f"best: dat={currmode}, score={model.best_score_:0.5f}, model={GetBestModelCTOR(model.estimator,model.best_params_)}", model.best_estimator_ 

def ClassificationReport(model, X_test, y_test, target_names=None):
    assert X_test.shape[0]==y_test.shape[0]
    print("\nDetailed classification report:")
    print("\tThe model is trained on the full development set.")
    print("\tThe scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, model.predict(X_test)                 
    print(classification_report(y_true, y_pred, target_names=target_names))
    print()
    
def FullReport(model, X_test, y_test, t):
    print(f"SEARCH TIME: {t:0.2f} sec")
    beststr, bestmodel = SearchReport(model)
    ClassificationReport(model, X_test, y_test)    
    print(f"CTOR for best model: {bestmodel}\n")
    print(f"{beststr}\n")
    return beststr, bestmodel
    
# def LoadAndSetupData(mode, test_size=0.3):
#     assert test_size>=0.0 and test_size<=1.0
    
#     def ShapeToString(Z):
#         n = Z.ndim
#         s = "("
#         for i in range(n):
#             s += f"{Z.shape[i]:5d}"
#             if i+1!=n:
#                 s += ";"
#         return s+")"

#     global currmode
#     currmode=mode
#     print(f"DATA: {currmode}..")
    
#     if mode=='moon':
#         X, y = itmaldataloaders.MOON_GetDataSet(n_samples=5000, noise=0.2)
#         itmaldataloaders.MOON_Plot(X, y)
#     elif mode=='mnist':
#         X, y = itmaldataloaders.MNIST_GetDataSet(load_mode=0)
#         if X.ndim==3:
#             X=np.reshape(X, (X.shape[0], -1))
#     elif mode=='iris':
#         X, y = itmaldataloaders.IRIS_GetDataSet()
#     else:
#         raise ValueError(f"could not load data for that particular mode='{mode}', only 'moon'/'mnist'/'iris' supported")
        
#     print(f'  org. data:  X.shape      ={ShapeToString(X)}, y.shape      ={ShapeToString(y)}')

#     assert X.ndim==2
#     assert X.shape[0]==y.shape[0]
#     assert y.ndim==1 or (y.ndim==2 and y.shape[1]==0)    
    
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=test_size, random_state=0, shuffle=True
#     )
    
#     print(f'  train data: X_train.shape={ShapeToString(X_train)}, y_train.shape={ShapeToString(y_train)}')
#     print(f'  test data:  X_test.shape ={ShapeToString(X_test)}, y_test.shape ={ShapeToString(y_test)}')
#     print()
    
#     return X_train, X_test, y_train, y_test

def TryKerasImport(verbose=True):
    
    kerasok = True
    try:
        import keras as keras_try
    except:
        kerasok = False

    tensorflowkerasok = True
    try:
        import tensorflow.keras as tensorflowkeras_try
    except:
        tensorflowkerasok = False
        
    ok = kerasok or tensorflowkerasok
    
    if not ok and verbose:
        if not kerasok:
            print("WARNING: importing 'keras' failed", file=sys.stderr)
        if not tensorflowkerasok:
            print("WARNING: importing 'tensorflow.keras' failed", file=sys.stderr)

    return ok
    
print(f"OK(function setup" + ("" if TryKerasImport() else ", hope MNIST loads works because it seems you miss the installation of Keras or Tensorflow!") + ")")

OK(function setup)


In [3]:
from sklearn.utils import shuffle
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Data information
print("Data Shapes:")
print(f"x_train: {X_train.shape} | y_train: {y_train.shape}")
print(f"x_test: {X_test.shape} | y_test: {y_test.shape}\n")

# Shuffle data
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

# Enhanced CNN model with regularization
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),  # Added dropout for regularization
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),  # Increased dropout
    
    layers.Conv2D(128, (3, 3), activation='relu'),  # Added more filters
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),  # Increased units
    layers.Dropout(0.5),  # Higher dropout for dense layer
    layers.Dense(1, activation='sigmoid')
])

# Compile with lower learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# Early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train model with validation
print("Training model...\n")
history = model.fit(
    X_train, y_train,
    epochs=30,  # Increased epochs
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

model.summary()

# Evaluate model
print("\nModel Evaluation:")
test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test)
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f'\nTest accuracy: {test_acc:.4f}')
print(f'Test precision: {test_precision:.4f}')
print(f'Test recall: {test_recall:.4f}')
print(f'Test F1-score: {test_f1:.4f}')

# Generate predictions
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype("int32")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, 
                          target_names=['Not Ice Cream', 'Ice Cream']))

# Enhanced confusion matrix plot
plt.figure(figsize=(10, 8))
conf_mat = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Not Ice Cream', 'Ice Cream'],
            yticklabels=['Not Ice Cream', 'Ice Cream'],
            annot_kws={"size": 14})
plt.title('Confusion Matrix', fontsize=16)
plt.ylabel('True Label', fontsize=14)
plt.xlabel('Predicted Label', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

# Enhanced training history plots
plt.figure(figsize=(14, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation', linewidth=2)
plt.title('Model Accuracy', fontsize=14)
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation', linewidth=2)
plt.title('Model Loss', fontsize=14)
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Save model
model.save('ice_cream_classifier_improved.h5')
print("\nModel saved as 'ice_cream_classifier_improved.h5'")

Data Shapes:
x_train: (1800, 512, 512, 3) | y_train: (1800,)
x_test: (200, 512, 512, 3) | y_test: (200,)



c:\Users\olive\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training model...

Epoch 1/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 53s 911ms/step - accuracy: 0.5610 - loss: 1.3720 - precision: 0.5708 - recall: 0.5419 - val_accuracy: 0.7000 - val_loss: 0.6130 - val_precision: 0.6471 - val_recall: 0.8800
Epoch 2/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 51s 886ms/step - accuracy: 0.6915 - loss: 0.5831 - precision: 0.7005 - recall: 0.6602 - val_accuracy: 0.7100 - val_loss: 0.5901 - val_precision: 0.6567 - val_recall: 0.8800
Epoch 3/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 51s 886ms/step - accuracy: 0.6942 - loss: 0.5865 - precision: 0.6910 - recall: 0.6764 - val_accuracy: 0.7600 - val_loss: 0.5663 - val_precision: 0.7203 - val_recall: 0.8500
Epoch 4/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 50s 880ms/step - accuracy: 0.7107 - loss: 0.5600 - precision: 0.7014 - recall: 0.7388 - val_accuracy: 0.7800 - val_loss: 0.5571 - val_precision: 0.7642 - val_recall: 0.8100
Epoch 5/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 50s 875ms/step - accuracy: 0.6833 - loss: 0.5815 - precision: 0.6914 - recall: 0.6968 - val_accurac

KeyboardInterrupt: 

In [ ]:
# !pip install scikeras
# !pip install --upgrade keras
# !pip install --upgrade tensorflow
!pip show tensorflow

Name: tensorflow
Version: 2.19.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\hassa\anaconda3\Lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing-extensions, wrapt
Required-by: 


In [ ]:
# from scikeras.wrappers import KerasClassifier
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.metrics import classification_report, confusion_matrix
# from time import time
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# import tensorflow as tf

# IMG_SIZE = 512

# def build_model(activation='relu'):
#     model = Sequential([
#         Conv2D(32, (3, 3), activation=activation, input_shape=(IMG_SIZE, IMG_SIZE, 3)),
#         MaxPooling2D((2, 2)),
#         Conv2D(64, (3, 3), activation=activation),
#         MaxPooling2D((2, 2)),
#         Conv2D(64, (3, 3), activation=activation),
#         Flatten(),
#         Dense(64, activation=activation),
#         Dense(1, activation='sigmoid')  
#     ])
#     return model

# param_grid = {
#     "batch_size": [16, 32],
#     "epochs": [5, 10],
#     "model__optimizer": ['adam', 'sgd'],
#     "model__activation": ['relu', 'tanh']
# }
# # For at forbedre vores model kan vi tune dens hyperparametre,
# # så jeg vil lige kort forklare hvad de forskellige parametre gør 
# # og hvad det er vi vælger imellem. Vi optimerer de følgende 4 parametre:
# # batch_size afgør størrelsen på de "batches" af billeder som vi fodrer modellen ad gangen, før den opdaterer weights.
# # epochs er hvor mange gange vi vil træne modellen på hele datasættet.
# # optimizer er den algoritme vi bruger til at opdatere weights i modellen.
# # activation er den funktion vi bruger til at aktivere neuronerne i modellen. 


# random_tuned = RandomizedSearchCV(
#     estimator=keras_clf,
#     param_distributions=param_grid,
#     n_iter=5,
#     scoring='accuracy',
#     cv=3,
#     verbose=1,
#     random_state=42
# )

# # Vores RandomizedSearch har også CrossValidation (CV), som bruges til at se hvor god vores model er på forskellige datasæt.
# # Cross Validation betyder at vi deler vores data op i flere dele og træner modellen på nogle af dem, for så at teste den på de andre dele.
# # Det hjælper os med at undgå overfitting, så den ikke generaliserer alt for hårdt på træningsdataen. 
# # I det her tilfælde bruger vi "3-fold cross validation", hvor den deler dataen op i 3 dele, hvor 2 af 
# # delene bliver brugt til at træne på, og den sidste tredjedel bliver brugt til at teste på. 
# # Den cycler så igennem de 3 dele så alle folds har prøvet at være "test" folden mindst 1 gang.


# start = time()
# random_tuned.fit(X_train, y_train)  
# t = time() - start

# # Hyper parameter search
# # En måde at forbedre vores model på er at fintune de hyperparametre vi giver til modellen.
# # Det kan man gøre med en search, hvor man træner modellen med forskellige kombinationer af hyperparametre,
# # og ser hvilken kombination der giver den bedste score med laveste loss.


# y_pred = random_tuned.predict(X_test)
# print("\n--- Classification Report ---")
# print(classification_report(y_test, y_pred))
# print("\n--- Confusion Matrix ---")
# print(confusion_matrix(y_test, y_pred))
# print(f"\nBest Parameters: {random_tuned.best_params_}")
# print(f"Search took {t:.2f} seconds.")


ImportError: cannot import name 'float8_e4m3b11fnuz' from 'tensorflow.python.framework.dtypes' (c:\Users\morte\anaconda3\Lib\site-packages\tensorflow\python\framework\dtypes.py)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

IMG_SIZE = 512

# Build the model with your specified parameters
def build_model(activation='relu'):
    model = Sequential([
        Conv2D(32, (3, 3), activation=activation, input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation),
        Flatten(),
        Dense(64, activation=activation),
        Dense(1, activation='sigmoid')  
    ])
    # Compile with SGD optimizer
    model.compile(optimizer=SGD(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Instantiate the model
best_model = build_model(activation='relu')

# Train it with your specified settings
best_model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)



Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.5582 - loss: 0.9003 - val_accuracy: 0.1139 - val_loss: 0.7846
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.6640 - loss: 0.6202 - val_accuracy: 0.0611 - val_loss: 1.2733
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.7121 - loss: 0.5701 - val_accuracy: 0.8806 - val_loss: 0.4781
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.7190 - loss: 0.5596 - val_accuracy: 0.6111 - val_loss: 0.6560
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.7377 - loss: 0.5411 - val_accuracy: 0.6861 - val_loss: 0.5812
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step

Er det is?: Ja (score: 0.8621)


In [ ]:
# Load and preprocess the new image
img_path = "ny_is.jpg"
new_img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
new_img_array = img_to_array(new_img) / 255.0  # normalize just like training data
new_img_array = np.expand_dims(new_img_array, axis=0)  # shape (1, IMG_SIZE, IMG_SIZE, 3)

# Load and preprocess the new image
hund_img_path = "hund.png"
hund_img = load_img(hund_img_path, target_size=(IMG_SIZE, IMG_SIZE))
hund_img_array = img_to_array(hund_img) / 255.0  # normalize just like training data
hund_img_array = np.expand_dims(hund_img_array, axis=0)  # shape (1, IMG_SIZE, IMG_SIZE, 3)

# Load hakkeboof
h_img_path = "hakkebof.jpg"
h_img = load_img(h_img_path, target_size=(IMG_SIZE, IMG_SIZE))
h_img_array = img_to_array(h_img) / 255.0 
h_img_array = np.expand_dims(h_img_array, axis=0)


# Make a prediction
prediction = best_model.predict(new_img_array)
hund_prediction = best_model.predict(hund_img_array)
hakkeboof_prediction = best_model.predict(h_img_array)

# Interpret result
is_ice_cream = prediction[0][0] >= 0.5  # threshold for binary classification
is_hund_ice_cream = hund_prediction[0][0] >= 0.5 
is_hakkeboof_ice_cream = hakkeboof_prediction[0][0] >= 0.5 

print(f"\nEr det is?: {'Ja' if is_ice_cream else 'Nej'} (score: {prediction[0][0]:.4f})")

# Se om den genkender hunde
print(f"\nEr hunden is?: {'Ja' if is_hund_ice_cream else 'Nej'} (score: {hund_prediction[0][0]:.4f})")

# Se om den genkender hakkeboof
print(f"\nEr hakkeboof is?: {'Ja' if is_hakkeboof_ice_cream else 'Nej'} (score: {hakkeboof_prediction[0][0]:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

Er det is?: Ja (score: 0.8621)

Er hunden is?: Nej (score: 0.1821)
